In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('heart.csv') 
data.isna().sum()

age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64

In [2]:
data

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [3]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.svm import SVC

classifier = SVC()

In [7]:
from sklearn.model_selection import GridSearchCV

parameters = [
    {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']},
    {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},
    {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['sigmoid'], 'gamma': ['scale'], 'coef0': [0.0]}
]

grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)

grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print("Best Accuracy: {:.2f} %".format(best_accuracy * 100))
print("Best Parameters:", best_parameters)

Best Accuracy: 82.67 %
Best Parameters: {'C': 0.5, 'gamma': 0.1, 'kernel': 'rbf'}


In [8]:
cv_results = grid_search.cv_results_
for mean_score, std_score, params in zip(cv_results["mean_test_score"], cv_results["std_test_score"], cv_results["params"]):
    print(f"Mean: {mean_score:.4f}, STD: {std_score:.4f} for {params}")

Mean: 0.8058, STD: 0.0644 for {'C': 0.25, 'kernel': 'linear'}
Mean: 0.8018, STD: 0.0604 for {'C': 0.5, 'kernel': 'linear'}
Mean: 0.7978, STD: 0.0643 for {'C': 0.75, 'kernel': 'linear'}
Mean: 0.7978, STD: 0.0643 for {'C': 1, 'kernel': 'linear'}
Mean: 0.8140, STD: 0.0621 for {'C': 0.25, 'gamma': 0.1, 'kernel': 'rbf'}
Mean: 0.8098, STD: 0.0562 for {'C': 0.25, 'gamma': 0.2, 'kernel': 'rbf'}
Mean: 0.6073, STD: 0.0505 for {'C': 0.25, 'gamma': 0.3, 'kernel': 'rbf'}
Mean: 0.5537, STD: 0.0243 for {'C': 0.25, 'gamma': 0.4, 'kernel': 'rbf'}
Mean: 0.5413, STD: 0.0090 for {'C': 0.25, 'gamma': 0.5, 'kernel': 'rbf'}
Mean: 0.5413, STD: 0.0090 for {'C': 0.25, 'gamma': 0.6, 'kernel': 'rbf'}
Mean: 0.5413, STD: 0.0090 for {'C': 0.25, 'gamma': 0.7, 'kernel': 'rbf'}
Mean: 0.5413, STD: 0.0090 for {'C': 0.25, 'gamma': 0.8, 'kernel': 'rbf'}
Mean: 0.5413, STD: 0.0090 for {'C': 0.25, 'gamma': 0.9, 'kernel': 'rbf'}
Mean: 0.8267, STD: 0.0572 for {'C': 0.5, 'gamma': 0.1, 'kernel': 'rbf'}
Mean: 0.7975, STD: 0.0573 f

In [10]:
import joblib

best_model = grid_search.best_estimator_
joblib.dump(best_model, 'svm_heart_disease_model.pkl')

['svm_heart_disease_model.pkl']